### OLS Task01
#### 쇼핑 고객 데이터

- Customer ID: 고객 아이디
- Gender: 고객의 성별
- Age: 고객의 나이
- Annual Income: 고객의 연소득
- Spending Score: 고객 행동 및 지출 성격에 따라 상점에서 할당한 점수
- Profession: 직업, 전문직
- Work Experience: 고객의 근무 경력(연 단위)
- Family Size: 가족 구성원 수

#### Target
- Spending Score: 고객 별 소비 점수

In [1]:
import pandas as pd

c_df = pd.read_csv('./datasets/customers.csv')

c_df

,CustomerID,Gender,Age,Annual Income ($),Spending Score (1-100),Profession,Work Experience,Family Size
0,1,Male,19,15000,39,Healthcare,1,4
1,2,Male,21,35000,81,Engineer,3,3
2,3,Female,20,86000,6,Engineer,1,1
3,4,Female,23,59000,77,Lawyer,0,2
4,5,Female,31,38000,40,Entertainment,2,6
...,...,...,...,...,...,...,...,...
1995,1996,Female,71,184387,40,Artist,8,7
1996,1997,Female,91,73158,32,Doctor,7,7
1997,1998,Male,87,90961,14,Healthcare,9,2
1998,1999,Male,77,182109,4,Executive,7,2


#### 회귀 분석
- 수치형, 범주형 (제일 먼저 확인)
- 타겟 데이터 분포
- 상관관계 corr()
- OLS
- VIF
- One-Hot 인코딩 지양
- Label 인코딩 순서에 맞게 전처리
- (x1 * W1 + x2 * w2) * w_category = new_feature  
  상관성이 있는 데이터끼리 feature engineering (되면 할 것)

#### 범주형 데이터들
- Gender, Profession
> - 데이터프레임 복사 후, 복사본에서 제거
> - 복사본 전처리 끝나면 원본에서 가져와 붙이기